In [4]:
import re
import os
import pathlib
from typing import List, Dict


import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib import gridspec
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
sns.set_theme(style="darkgrid")

In [5]:
experiment = "03_http_payload"

# Root of the project
project_dir = pathlib.Path().resolve().parent

# Directory containing experiment results
results_dir = (pathlib.Path(project_dir) / "results" / experiment).resolve()
results_dir


PosixPath('/home/richard/projects/mesh-bench/results/03_http_payload')

In [6]:
"""
Clean the obtained results from the experiments
- Remove unnecessary columns
- Extract metadata from filenames
- Construct and clean data into a pandas.Series object
- Convert data measures in seconds to ms
"""

# Filter columns
required_cols = [
    "StartTime",
    "RequestedQPS",
    "ActualQPS",
    "DurationHistogram",
]


# fortio reports in seconds, convert to miliseconds
multiplier = 1000


# Regex to extract metadata
# 0 -> Full match
# 1 -> Protocol (http/grpc)
# 2 -> Mesh
# 3 -> Requested QPS
# 4 -> Transfer in bytes
# 5 -> Repetitionct dimensions from the filename
name_re = re.compile("^([a-z]+)_([a-z]+)_(\d+|MAX)_(\d+)_(\d+).*json$")

"""
Reads a fortio results JSON file, cleans the data and returns
it as a pandas.Series
"""
def clean_fortio_results(results_file: pathlib.Path) -> pd.Series :
    # Read the JSON file as pandas series
    s = pd.read_json(results_file, typ="series")

    # Filter unnecesary columns
    s = s[required_cols]

    # Extract metadata from filename and add it to the series
    matches = re.match(name_re, str(results_file.name))
    s["mesh"] = matches[2]
    s["payload"] = matches[4]

    # Extract nested columns in 'DurationHistogram'
    for key, val in s["DurationHistogram"].items():
        s[key] = val
    s = s.drop(["DurationHistogram", "Percentiles"])

    # Convert obervations from seconds to miliseconds
    s.loc[["Min", "Max", "Sum", "Avg"]] *= multiplier

    # Construct Histogram Bins, Counts and Percentiles
    data = s["Data"]

    s["counts"] = [x["Count"] for x in data]
    s["percentages"] = [x["Percent"] for x in data]

    # Bins (default pd.cut function makes them as categoricals of intervals)
    # https://pandas.pydata.org/docs/reference/api/pandas.Categorical.html
    # https://pandas.pydata.org/docs/reference/api/pandas.Interval.html
    s["categories"]= pd.Categorical([pd.Interval(x["Start"] * multiplier , x["End"] * multiplier) for x in data])
    s["bins"] = [x.left for x in s["categories"]]
    s["bins"].append(s["categories"][-1].right)

    s = s.drop("Data")

    return s


In [7]:
"""
Parse all experiment results:
- Read the JSON files as a flat data structure
- Parse each result file as a individual data frame
- Concatenate all data frames where each row represents an observation
"""

series = []
for root, _, files in os.walk(results_dir, topdown=False):

    print(f"Parsing Experiment results:")
    print(f"Exp: {experiment}\nResults: {len(files)}")

    for file in files:
        # Skip resource usage files
        if not re.match("^(http|grpc).*\.json$", file):
            continue

        # Absolute path of the results file
        p = (pathlib.Path(root) / file).resolve()

        res = clean_fortio_results(p)
        series.append(res)

df = pd.DataFrame(series)

# Re-order
df = df[[
    "mesh",
    "RequestedQPS",
    "payload",
    "ActualQPS",
    "StartTime",
    "Count",
    "Min",
    "Max",
    "Sum",
    "Avg",
    "StdDev",
    "counts",
    "percentages",
    "categories",
    "bins",
]]

# Sort
df = df.sort_values(by=["mesh", "RequestedQPS", "payload"], ignore_index=True)
df.head()

Parsing Experiment results:
Exp: 03_http_payload
Results: 41


,mesh,RequestedQPS,payload,ActualQPS,StartTime,Count,Min,Max,Sum,Avg,StdDev,counts,percentages,categories,bins
0,baseline,100,0,99.965405,2022-06-26T15:47:27.590351364Z,89984,0.127398,108.811639,38636.269136,0.429368,0.002438,"[499, 58410, 26130, 2761, 430, 192, 134, 109, ...","[0.554543029871977, 65.46608285917496, 94.5045...","[(0.127398, 0.2], (0.2, 0.30000000000000004], ...","[0.127398, 0.2, 0.30000000000000004, 0.4, 0.5,..."
1,baseline,100,1000,99.965474,2022-06-26T16:02:28.080492071Z,89984,0.142442,167.979056,41815.321141,0.464697,0.002715,"[149, 49446, 33965, 4062, 575, 199, 134, 88, 7...","[0.16558499288762402, 55.115353840682786, 92.8...","[(0.142442, 0.2], (0.2, 0.30000000000000004], ...","[0.142442, 0.2, 0.30000000000000004, 0.4, 0.5,..."
2,baseline,100,10000,99.965401,2022-06-26T16:17:28.569445715Z,89984,0.174126,177.480738,44267.976370,0.491954,0.002941,"[54, 35749, 45379, 5721, 1070, 305, 143, 126, ...","[0.06001066856330001, 39.78818456614509, 90.21...","[(0.174126, 0.2], (0.2, 0.30000000000000004], ...","[0.174126, 0.2, 0.30000000000000004, 0.4, 0.5,..."
3,cilium,100,0,99.965400,2022-06-27T12:51:53.876748378Z,89984,0.127580,122.002537,45854.109841,0.509581,0.002243,"[11, 5742, 59408, 17820, 3803, 875, 348, 224, ...","[0.012224395448079002, 6.393358819345662, 72.4...","[(0.12758, 0.2], (0.2, 0.30000000000000004], (...","[0.12758, 0.2, 0.30000000000000004, 0.4, 0.5, ..."
4,cilium,100,1000,99.965455,2022-06-27T13:06:54.457697549Z,89984,0.168528,170.908343,47969.343397,0.533087,0.002551,"[6, 4368, 58007, 19913, 4236, 1009, 367, 208, ...","[0.006667852062588001, 4.860864153627311, 69.3...","[(0.168528, 0.2], (0.2, 0.30000000000000004], ...","[0.168528, 0.2, 0.30000000000000004, 0.4, 0.5,..."


In [8]:
"""
Plotting of the results
- Create grid of subplots
- Plot figures on logarithmic scale (because of outliers)
- Plot Constant Throughput per mesh
"""


def plot_http_payload_results(mesh: str, data:pd.DataFrame, axes: List[matplotlib.axes]) -> matplotlib.axes:
    # Plot each row as subplot
    for i ,(_, row) in enumerate(data.iterrows()):
        ax = axes[i]

        # Plot data
        binedges = np.array(row["bins"])
        weights = np.array(row["counts"])
        x = (binedges[1:] + binedges[:-1]) / 2.0

        sns.histplot(data=row, x=x, bins=binedges, weights=weights, color="b", ax=ax)

        # Labels
        plot_title = (
            f"$\\bf{mesh}$\n"
            f"Payload Size: {row['payload']}b\n"
            f"min: {row['Min']:.1f} ms, avg: {row['Avg']:.1f} ms, max: {row['Max']:.1f} ms"
        )
        ax.set_ylabel("Request Count")
        ax.set_xlabel("Latency in ms")
        ax.set_title(plot_title, y=1.05)


        # Y AXIS
        ax.yaxis.set_major_formatter(ticker.ScalarFormatter())


        # X AXIS
        ax.set_xscale("log")
        ax.xaxis.set_major_formatter(ticker.ScalarFormatter())


        # # Second plot with cumulative density
        ax2 = ax.twinx()
        ax2.set_alpha(0.5)
        ax2.grid(False)
        sns.ecdfplot(data=row, x=x, weights=weights, color="orange", ax=ax2)

    fig.tight_layout()
    return fig

TypeError: Parameters to generic types must be types. Got <module 'matplotlib.axes' from '/home/richard/.local/lib/python3.8/site-packages/matplotlib/axes/__i.

In [ ]:
# Create grid of plots 
fig, axes = plt.subplots(nrows=5, ncols=3, figsize=(15, 25))

# Set figure title
fig.suptitle(f"HTTP Payload", fontsize=18, y=1)

meshes = df["mesh"].unique()
meshes.sort()
meshes

for i, m in enumerate(meshes):
    # Subset dataframe
    data = df[df["mesh"] == m]

    plot_http_payload_results(mesh=m, data=data, axes=axes[i])

NameError: name 'plt' is not defined